## Import libraries

In [ ]:
import os
from datetime import datetime, date, timedelta
import pandas as pd
import ee

## User inputs

In [ ]:
from cropengine.models import get_available_models
from cropengine.crop import get_available_crops, get_available_crop_varieties

# Location
latitude = 52.53
longitude = 14.12

# Crop information
campaign_start_date = "2006-01-01"
crop_start_date = "2006-04-05"
crop_start_type = "emergence"

crop_end_date = "2006-10-20"
crop_end_type = "harvest"
campaign_end_date = "2007-01-01"

max_duration = 300

# Other inputs
# Get the available model info (**To be displayed in the dashboard)
models_info = get_available_models()
model = "Wofost72_WLP_CWB"  # User can select from 'models_info' dropdown

crops_info = get_available_crops(model)
crop_name = "sugarbeet"  # User can select from 'crops_info' dropdown

crop_varieties_info = get_available_crop_varieties(
    model, crop_name
)  # User can select from 'crop_varieties_info' dropdown
crop_variety = "Sugarbeet_601"  # User can select from 'crop_varieties_info' dropdown

## Create workspace directory

In [ ]:
workspace_dir = os.path.join(".", "workspace")  # To be decided by Jahid
os.makedirs(workspace_dir, exist_ok=True)

## Data and parameters preparation

### Weather

In [ ]:
from cropengine.weather import GEEWeatherDataProvider

# ee.Authenticate()
# Prepare weather data for a specific point
from datetime import datetime, timedelta


def make_inclusive(end_date_str: str) -> str:
    """Convert string to date, add 1 day, return string."""
    d = datetime.strptime(end_date_str, "%Y-%m-%d").date()
    d_inc = d + timedelta(days=1)
    return d_inc.strftime("%Y-%m-%d")


campaign_end_date_inclusive = make_inclusive(campaign_end_date)
meteo_data = GEEWeatherDataProvider(
    start_date=campaign_start_date,
    end_date=campaign_end_date_inclusive,
    latitude=latitude,
    longitude=longitude,
    filepath=os.path.join(workspace_dir, "meteo.xlsx"),
)
meteo_data.save_weather_excel()

### Soil

In [ ]:
from cropengine.soil import IsricSoilDataProvider, WOFOSTSoilParameterProvider

# Prepare soil data for a specific point
soil_data = IsricSoilDataProvider(
    latitude=latitude,
    longitude=longitude,
    depths="0-5cm",
    values=["mean"],
    filepath=os.path.join(workspace_dir, "soil.csv"),
)
# soil_data = soil_data.get_data()
soil_data = pd.read_csv(
    "/beegfs/halder/GITHUB/LEARNING/cropengine/docs/examples/workspace/soil.csv"
)
soil_data = soil_data[soil_data["metric"] == "mean"]

# Extract soil parameters from the soil data
soil_params_provider = WOFOSTSoilParameterProvider(soil_data)
soil_params = soil_params_provider.get_params()

In [ ]:
soil_params_info = (
    soil_params_provider.param_metadata
)  # User can change parameters' value from 'soil_params_info' table
soil_params_info = (
    pd.DataFrame(soil_params_info).sort_values(by="parameter").reset_index(drop=True)
)
print(soil_params_info.shape)
soil_params_info

### Site

In [ ]:
from cropengine.site import WOFOSTSiteParametersProvider

# Extract site parameters
site_params_provider = WOFOSTSiteParametersProvider(model, WAV=10)
site_params = site_params_provider.get_params()

In [ ]:
site_params_info = (
    site_params_provider.param_metadata
)  # User can change parameters' value from 'site_params_info' table
site_params_info = (
    pd.DataFrame(site_params_info).sort_values(by="parameter").reset_index(drop=True)
)
print(site_params_info.shape)
site_params_info

### Crop

In [ ]:
from cropengine.crop import WOFOSTCropParametersProvider

# Extract crop parameters
crop_params_provider = WOFOSTCropParametersProvider(
    crop_name=crop_name, variety_name=crop_variety
)
print(crop_params_provider)

In [ ]:
crop_params_info = (
    crop_params_provider.param_metadata
)  # User can change parameters' value from 'crop_params_info' table
crop_params_info = (
    pd.DataFrame(crop_params_info).sort_values(by="parameter").reset_index(drop=True)
)
print(crop_params_info.shape)
crop_params_info

### Packing all parameters

In [ ]:
from pcse.base import ParameterProvider

crop_dict = dict(zip(crop_params_info["parameter"], crop_params_info["default"]))
soil_params_info = soil_params_info.dropna()
soil_dict = dict(zip(soil_params_info["parameter"], soil_params_info["value"]))
site_dict = site_params.copy()

parameters = ParameterProvider(
    cropdata=crop_dict, soildata=soil_dict, sitedata=site_dict
)

## Define agromanagement

### Create irrigation timed events

In [ ]:
from cropengine.agromanagement import WOFOSTAgroEventBuilder

agro_event_builder = WOFOSTAgroEventBuilder()

# Build timed events (irrigation)
irrigation_schedule = [
    {"event_date": "2006-05-25", "amount": 3.0, "efficiency": 0.7},
    {"event_date": "2006-06-30", "amount": 2.5, "efficiency": 0.7},
]

irrigation_events = agro_event_builder.create_timed_events(
    signal_type="irrigate", events_list=irrigation_schedule
)

# Build state Events (fertilization based on DVS)
nitrogen_schedule = [
    {"threshold": 0.3, "N_amount": 40, "N_recovery": 0.7},
    {"threshold": 0.6, "N_amount": 60, "N_recovery": 0.7},
    {"threshold": 1.12, "N_amount": 40, "N_recovery": 0.7},
]

nitrogen_events = agro_event_builder.create_state_events(
    signal_type="apply_n",
    state_var="DVS",
    zero_condition="rising",
    events_list=nitrogen_schedule,
)

## Create agromanagement with timed events

In [ ]:
from cropengine.agromanagement import WOFOSTAgroManagementProvider

agro_management_provider = WOFOSTAgroManagementProvider()

# Add campaign with events to provider
agro_management_provider.add_campaign(
    campaign_start_date=campaign_start_date,
    campaign_end_date=campaign_end_date,
    crop_name=crop_name,
    variety_name=crop_variety,
    crop_start_date=crop_start_date,
    crop_end_date=crop_end_date,
    crop_start_type=crop_start_type,
    crop_end_type=crop_end_type,
    max_duration=max_duration,
    timed_events=[irrigation_events],  # Note: this must be a list of event objects
    state_events=[nitrogen_events],  # Note: this must be a list of event objects
)
agro_management_provider.add_trailing_empty_campaign()
# Save to YAML
agro_management_provider.save_to_yaml(os.path.join(workspace_dir, "agro.yaml"))
print("YAML file saved successfully.")

In [ ]:
from pcse.input import YAMLAgroManagementReader

agromanagement = YAMLAgroManagementReader(os.path.join(workspace_dir, "agro.yaml"))
print(agromanagement)

## Load the weather data

In [ ]:
from pcse.input import ExcelWeatherDataProvider

weatherfile = os.path.join(workspace_dir, "meteo.xlsx")
weatherdataprovider = ExcelWeatherDataProvider(weatherfile)
print(weatherdataprovider)

## Run the model

In [ ]:
from pcse.models import Wofost72_WLP_CWB

wofsim = Wofost72_WLP_CWB(parameters, weatherdataprovider, agromanagement)

In [ ]:
wofsim.run_till_terminate()
df_results = pd.DataFrame(wofsim.get_output())
df_results = df_results.set_index("day")
df_results.tail()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(12, 12), sharex=True)
axes = axes.flatten()

for i, var in enumerate(["DVS", "TAGP", "LAI", "SM", "TWSO"]):
    sns.lineplot(x=df_results.index, y=df_results[var], ax=axes[i])

plt.tight_layout()
plt.show()